In [1]:
import os
os.environ["FLASH_ATTWNTION_FORCE_DISABLED"]= "1"
os.environ["DISABLE_TRITION"]= "1"

In [2]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 1.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.6/275.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.8/145.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 59.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 MB 10.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 4.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 112.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 82.5 MB/s eta 0:00:0

In [3]:
!pip install transformers==4.51.3 trl==0.8.6 bitsandbytes accelerate --no-deps --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 5.2 MB/s eta 0:00:00a 0:00:01


In [4]:
!nvidia-smi

Fri May 30 00:50:51 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P0             26W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer

from huggingface_hub import login 
from transformers import TrainingArguments
from datasets import load_dataset
import wandb 

from kaggle_secrets import UserSecretsClient


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-05-30 00:51:06.963949: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748566267.151514      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748566267.206678      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


In [6]:
import pandas as pd
from datasets import Dataset

dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT","en")

df = pd.DataFrame(dataset["train"])

print("Columns:", df.columns)
print(df.head(5))

README.md:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

medical_o1_sft.json:   0%|          | 0.00/58.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/19704 [00:00<?, ? examples/s]

Columns: Index(['Question', 'Complex_CoT', 'Response'], dtype='object')
                                            Question  \
0  Given the symptoms of sudden weakness in the l...   
1  A 33-year-old woman is brought to the emergenc...   
2  A 61-year-old woman with a long history of inv...   
3  A 45-year-old man with a history of alcohol us...   
4  A 45-year-old man presents with symptoms inclu...   

                                         Complex_CoT  \
0  Okay, let's see what's going on here. We've go...   
1  Okay, let's figure out what's going on here. A...   
2  Okay, let's think about this step by step. The...   
3  Alright, let’s break this down. We have a 45-y...   
4  Okay, so here's a 45-year-old guy who's experi...   

                                            Response  
0  The specific cardiac abnormality most likely t...  
1  In this scenario, the most likely anatomical s...  
2  Cystometry in this case of stress urinary inco...  
3  Considering the clinical presen

In [7]:
# def format_cot(example):
#     text = (
#         "<system>You are a medical expert...</system>\n"
#         f"<user>Question: {example['Question']}</user>\n"
#         f"<think>{example['Complex_CoT']}</think>\n"
#         f"<response>{example['Response']}</response>"
#     )
#     return {"text": text}

# formatted_dataset_train = dataset_train.map(format_cot)

def format_cot(example):
    text = (
        f"<user>Question: {example['Question']}</user>\n"
        f"<think>{example['Complex_CoT']}</think>\n"
        f"<response>{example['Response']}</response>"
    )
    return {"text": text}

formatted_data = df.apply(format_cot, axis=1)
formatted_df = pd.DataFrame(formatted_data.tolist())

In [8]:
val_df = formatted_df.sample(n=100 , random_state= 42)
train_df = formatted_df.drop(val_df.index)

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

print(train_dataset[0])


{'text': "<user>Question: Given the symptoms of sudden weakness in the left arm and leg, recent long-distance travel, and the presence of swollen and tender right lower leg, what specific cardiac abnormality is most likely to be found upon further evaluation that could explain these findings?</user>\n<think>Okay, let's see what's going on here. We've got sudden weakness in the person's left arm and leg - and that screams something neuro-related, maybe a stroke?\n\nBut wait, there's more. The right lower leg is swollen and tender, which is like waving a big flag for deep vein thrombosis, especially after a long flight or sitting around a lot.\n\nSo, now I'm thinking, how could a clot in the leg end up causing issues like weakness or stroke symptoms?\n\nOh, right! There's this thing called a paradoxical embolism. It can happen if there's some kind of short circuit in the heart - like a hole that shouldn't be there.\n\nLet's put this together: if a blood clot from the leg somehow travels 

In [9]:
from kaggle_secrets import UserSecretsClient
from unsloth import FastLanguageModel
from transformers import AutoTokenizer

user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("hf_token") 
wandb_token = user_secrets.get_secret("wnb")

import wandb
wandb.login(key="aef4b6a3e42d8c3a5c5d708b92fc26e9bd00db8b") 


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3.2-3B-Instruct",
    max_seq_length = 2048,
    dtype= None,
    load_in_4bit =True,
    token = hf_token,
)


FastLanguageModel.for_training(
    model, 
    use_gradient_checkpointing = True)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: muqadasejaz (muqadasejaz-pmas-arid-agriculture-university-rawalpindi) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


==((====))==  Unsloth 2025.5.9: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 6.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072, padding_idx=128004)
    (layers): ModuleList(
      (0): LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,)

In [10]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    lora_alpha = 32,
    lora_dropout = 0.0,
    bias = "none"
)

Unsloth 2025.5.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [11]:
def tokenize(example):
    tokenized = tokenizer(
        example["text"],
        truncation = True,
        padding = "max_length", 
        max_length = 2048
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

In [12]:
train_dataset = train_dataset.map(tokenize)
val_dataset = val_dataset.map(tokenize)

Map:   0%|          | 0/19604 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [13]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir = "llama3-finetuning", 
    per_device_train_batch_size = 2, 
    gradient_accumulation_steps = 2, 
    max_steps  = 100, 
    logging_steps = 1, 
    save_steps = 10,
    learning_rate = 2e-4,
    num_train_epochs = 1, 
    fp16 = True,
    optim = "adamw_torch", 
    lr_scheduler_type = "cosine", 
    warmup_steps = 5,
    report_to = "wandb",
    
)

In [14]:
print(val_df.columns.tolist())

['text']


In [15]:
print(val_df["text"].iloc[0])

<user>Question: A 24-year-old woman has progressively worsening episodes of severe, crampy abdominal pain, nonbloody diarrhea, mild abdominal distension, and a perianal fistula draining stool. Immunohistochemistry reveals dysfunction of the nucleotide oligomerization binding domain 2 (NOD2) protein. Which immunological protein is most likely overactive due to this dysfunction?</user>
<think>Hmm, a 24-year-old woman is experiencing these terrible stomach issues, like severe cramps and diarrhea, plus she's got a perianal fistula that's really uncomfortable. It sounds like a tough situation.

From what I know, these symptoms kinda fit the description of Crohn's disease, doesn't it? Crohn's is one of those inflammatory bowel diseases where the immune system kinda goes haywire.

Now, let's think about this NOD2 thing she's got going on. NOD2 is an important protein that helps recognize bacteria, like a bodyguard for our gut. If dysfunctional, it can lead to problems with detecting bacteria 

In [16]:
import re 
def extract_fields(text): 
     question_match = re.search(r"### Question:\n(.+?)\n### Reasoning:", text, re.DOTALL) 
     reasoning_match = re.search(r"### Reasoning:\n(.+?)\n### Answer:", text, re.DOTALL) 
     answer_match = re.search(r"### Answer:\n(.+)", text, re.DOTALL) 
     
     return {
        "Question": question_match.group (1).strip() if question_match else None, 
        "Complex_CoT": reasoning_match.group(1).strip() if reasoning_match else None, 
        "Response": answer_match.group (1).strip() if answer_match else None, 
     }
#Apply to all rows 
parsed_df = val_df["text"].apply(extract_fields).apply(pd.Series) 
#Merge with original dataframe if needed 
val_df = pd.concat((val_df, parsed_df), axis=1) 

In [17]:
def formatting_func(example): 
      question = example ["Question" ]
      reasoning = example ["Complex_CoT"] 
      response = example ["Response"]
      return f"### Question:\n {question} \n\n### Reasoning: \n{reasoning} \n\n### Answer: \n{response}" 

In [18]:
# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("unsloth/llama-3-8b-bnb-4bit", trust_remote_code=True)
# tokenizer.pad_token = tokenizer.eos_token  # set pad token if missing


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

In [18]:
from trl import SFTTrainer 

from unsloth.trainer import SFTTrainer  

trainer =  SFTTrainer( 
   model = model, 
   train_dataset= train_dataset, 
   args =  training_args, 
   tokenizer = tokenizer, 
   formatting_func= formatting_func, 
   packing=True, 
)

In [19]:
!pip install -q evaluate rouge_score

import evaluate 
rouge = evaluate.load("rouge")
#Reload fine-tuned model (if necessary) and run predictions again 
def generate_response_finetuned(example): 
    prompt = formatting_func(example) 
    inputs = tokenizer (prompt, return_tensors="pt", truncation=True, max_length=2848).to("cuda") 
    outputs = model.generate (**inputs, max_new_tokens=200) 
    return tokenizer.decode(outputs[0], skip_special_tokens=True) 
    
val_df["finetuned_pred"] = val_df.apply(generate_response_finetuned, axis=1) 
finetuned_scores =  rouge.compute(predictions=val_df["finetuned_pred"].tolist(), 
                                  references=val_df["Response"].tolist(),
                                  use_stemmer=True) 

print("ROUGE-L Before Fine-Tuning:", finetuned_scores["rougeL"])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.5 MB/s eta 0:00:00


AttributeError: 'NoneType' object has no attribute 'lower'

In [20]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 19,604 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 2 x 1) = 4
 "-____-"     Trainable parameters = 24,313,856/3,000,000,000 (0.81% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,9.632800
2,9.618600
3,9.200000
4,8.854300
5,7.336700
6,7.372800
7,6.730400
8,6.362800
9,5.909700
10,5.563300


TrainOutput(global_step=100, training_loss=4.917830023765564, metrics={'train_runtime': 2489.1056, 'train_samples_per_second': 0.161, 'train_steps_per_second': 0.04, 'total_flos': 1.39742146461696e+16, 'train_loss': 4.917830023765564, 'epoch': 0.020403999183840033})

In [ ]:
# #Reload fine-tuned model (if necessary) and run predictions again 
# def generate_response_finetuned (example): 
#     prompt = formatting_func(example) 
#     inputs = tokenizer (prompt, return_tensors="pt", truncation=True, max_length=2848).to("cuda") 
#     outputs = model.generate (**inputs, max_new_tokens=200) 
#     return tokenizer.decode(outputs[0], skip_special_tokens=True) 
# val_df["finetuned_pred"] = val_df.apply(generate_response_finetuned, axis=1) 
# finetuned_scores =  rouge.compute(predictions=val_df["finetuned_pred"].tolist(), 
#                                   references=val_df["Response"].tolist(),
#                                   use_stemmer=True) 

# print("ROUGE-L After Fine-Tuning:", finetuned_scores["rougeL"])

TypeError: 'Tensor' object is not callable

In [21]:
save_path = "finetune-llama"

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)


('finetune-llama/tokenizer_config.json',
 'finetune-llama/special_tokens_map.json',
 'finetune-llama/tokenizer.json')

In [22]:
from huggingface_hub import notebook_login

notebook_login()

In [23]:
model.push_to_hub("Muqadase/finetune-llama")
tokenizer.push_to_hub("Muqadase/finetune-llama")

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/97.3M [00:00<?, ?B/s]

Saved model to https://huggingface.co/Muqadase/finetune-llama


README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]